# Lab session 4: Data Warehousing and On-line Analytical Processing

## Introduction 

The aim of this lab is for you to gain experience with **Data Warehousing** and **On-line Analytical Processing (OLAP)** covered in lecture 5, and more specifically with the concepts of **data cubes**, **data cube measures**, **typical OLAP operations**, and **data cube computation**.

- This lab is the first part of a **two-lab assignment** that covers lectures 5 and 6, which is due on **Sunday, 3 July 2022, 11:59 PM**.
- The assignment will account for 10% of your overall grade. Questions in this lab sheet (Lab session 2) will contribute to 5% of your overall grade; questions in the sheet of Lab session 3 will count for another 5% of your overall grade.
- <font color = 'maroon'>The **assessed questions** are in the last section of this notebook (Assignment 1, part 2 of 2).</font> 

## Submission
- You are asked to submit: 
    - A **completed notebook**. That is, this notebook with your solutions to the assessed questions at the end.
    - A **report** with the answers to the assessed questions. The report should be in **PDF format** (so **NOT** *doc, docx, notebook* etc). It should be well identified with your name, student number, assignment number (for instance, Assignment 1), module, and marked with question numbers. 
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment1-StudentName-StudentNumber.pdf

## Important notes about the assignment: 
- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.
- For the below assignment questions, you can choose whether to answer them using pen-and-paper or using code. However, please note that questions which explicitly require loading CSV files or using a pre-loaded dataset must be addressed using code. A few assignment questions throughout the module might be easier to complete by hand, although could also be addressed using code. In both cases, either responding to questions manually or using code, please **do make sure to show your work** - i.e., how you derived the result, by showing the code that was used to generate the result that addresses the question and/or by writing down your reasoning or math derivations.
- Any mathematics included in the report must be written using Tex typesetting, or a system of comparable quality. A photograph of your handwritten derivations will not be accepted.

### How to approach this lab exercise
This notebook involves extensive use of a the Cubes package, which relies heavily on numerous abstractions and terminology that you may not be familiar with yet. This might be daunting at first, but do not be discouraged.

Make sure you have the reference at hand at all times: https://pythonhosted.org/cubes/index.html#developer-s-reference Whenever you are dealing with a specific function or class of the Cubes package, look it up in the reference to understand how it works.

You might also want to skim the logical model, but don't get stuck if there is something you don't understand. Just read on: https://pythonhosted.org/cubes/model.html 

You can also take a look at the rest of the documentation, including the tutorial (which this notebook draws plenty from): https://pythonhosted.org/cubes/index.html

**Please note** that the documentation for this project lacks polish. Nevertheless, it should help you to navigate the contents of this notebook and understand the functionalities of the package.


## 1. Introduction to Cubes

This section describes how to use Cubes (http://cubes.databrewery.org/), a lightweight Python framework for the development of reporting and analytical applications, Online Analytical Processing (OLAP), multidimensional analysis and aggregated data browsing. We will be working with v1.1 of Cubes.

Cubes also relies on methods from SQLAlchemy (https://www.sqlalchemy.org/), an open-source SQL toolkit and object-relational mapper for Python.

## 2. Data Preparation

The example data used here is the International Bank for Reconstruction and Development (IBRD) Balance Sheet. For this example we will be using the CSV file "IBRD_Balance_Sheet__FY2010.csv" which is provided among the materials for the lab. The CSV file includes records which are characterised by a Category (and subcategories), Line Item, Fiscal Year, and Amount (in US$ millions). We first start with imports:

In [ ]:
# If you are using Google Colab, you would need to run the lines below to install Cubes. 
# This might also be the case if you are running the notebook locally.
# You can comment the below line if you are running a local python installation with Cubes installed.
# The below line also installs a particular version of the sqlalchemy package needed to use Cubes.

#!pip install cubes
#!pip install -Iv sqlalchemy==1.3.9

from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

We can now load the data, create a table and populate it with contents of the CSV file. Note the categories and subcategories created; check the CSV file to understand the commands below.

To create the table, we employ a helper function provided in the Cubes package for training purposes.

In [ ]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

## 3. Creating a data cube

A *Workspace* object maintains the basic information and data structures we need. It contains cubes, maintains connections to the data stores, and more.

In [ ]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

You can check out the methods offered by *Workspace* to learn more about what it contains: https://pythonhosted.org/cubes/reference/workspace.html

For example, we can print information about the registered cubes as follows. Currently, the output will be empty. Try running the following cell again later, once you have created the first cube. 

In [ ]:
workspace.list_cubes()

The structure of data cubes (in terms of dimensions, measures, and aggregates) is specified in JSON files. We now import file 'tutorial_model.json' (found in the lab supplementary material) which includes an example model of the data cube, dimension tables, and aggregate functions for the CSV file we loaded previously.

In [ ]:
workspace.import_model("tutorial_model.json")

**Please make sure to inspect the structure of the above JSON file - this will be relevant for one of the assignment questions.**

We can now create a data cube based on the above data cube model and data table. Note that the argument matches the name of the database table we registered earlier.

In [ ]:
cube = workspace.cube("ibrd_balance")

## 4. Aggregations and OLAP operations

In Cubes, the functionalities for querying a data cube are implemented in an object called *Browser*. We can obtain a *Browser* for a cube as follows:

In [ ]:
browser = workspace.browser(cube)

We can now compute aggregates of the data cube as specified by the data cube model. For computing the total count of records:

In [ ]:
result = browser.aggregate()
result.summary["record_count"]

For computing a sum of the amount:

In [ ]:
result.summary["amount_sum"]

Now we can try to compute aggregates by year:

In [ ]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

Or compute aggregates by item category:

In [ ]:
result = browser.aggregate(drilldown=["item"])
for record in result:
    print(record)

The documentation for this function can be found here: https://pythonhosted.org/cubes/reference/browser.html?highlight=pointcut#result

We can also perform *slicing* and *dicing* operations on the data cube. The below example performs a slicing operation on the data cube by selecting only entries with the year being 2009, and displays aggregates according to the item category. Here, a *cell* defines a point of interest – portion of the cube to be aggregated or browsed.

In [ ]:
import cubes as cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

We can obtain the aggregates for the cube involved in the above operation as follows:

In [ ]:
result.summary

We can perform range cuts as well:

In [ ]:
cuts = [cubes.RangeCut("year", [2009], [2010])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

Compare the aggregates with the ones we obtained before.

In [ ]:
result.summary

Similarly, we can perform a *dicing* operation on the data cube by performing a selection on two or more dimensions. The below example selects entries with the year being 2009 and the item category being "a", and displays the aggregate results:

In [ ]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

**Question**: what is the effect of removing the `drilldown` argument in the `brower.aggregate` call above? Why?

We can also *drill down* lower in the Category hierarchy. Here, we perform a dicing operation to select records with year being 2009 and item category being "a" (corresponding to assets), and show aggregates for each subcategory level.

In [ ]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

## <font color = 'maroon'>Assignment 2, part 1 of 2</font>

Questions 1 is a pen-and-paper exercise; questions 2-3 are coding exercises.

Please **show your work** - i.e., show and explain your code/math, and write your reasoning.

1. A data warehouse for a music streaming company consists of the dimensions song, user, time (time and date of when the user listened to a song), and a measure called fee (fee paid by the streaming company to the artist every time a user listens to that song). 
    1. **[1 mark]** Draw a schema diagram for the above data warehouse using a star schema.
    2. **[0.5 marks]** Starting with the base cuboid [time, user, song], what specific OLAP operations should be performed in order to list the total fee collected for a given song for a given month of a given year (e.g. October 2021)?
    3. **[0.5 marks]** Assume that the time dimension has 3 levels: day, month, year; and that the song and user dimensions both have 1 level (not including the virtual level 'all'). How many cuboids will this cube contain (including the base and apex cuboids)?

1. **[1 mark]** Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your work in the PDF report. You can read the [Cubes package documentation](https://readthedocs.org/projects/cubes/downloads/pdf/latest/) for assistance in this task.

1. Consider the CSV file "country-income-large.csv":
   1. **[1 marks]** Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files you have created.
   2. **[1 mark]** Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 30 and 50.
  